# Annotation enhanced genetic fine-mapping

## Aim

This simulation study compares fine-mapping with use of annotations / without it, in terms of:

1. Improvements in fine-mapping resolution: with the use of annotations we expect to provide smaller sets of candidate SNPs than without them.
2. Improvements in power: the top signal from each candidate fine-mapping cluster is more likely to be the true causal signal when annotations are used.

## Simulation scheme

### Genotype data

We use real data genotypes from GTEx project, of ~600 European individual samples. We choose common variants (MAF > 1%). We create genomic regions (fine-mapping units) each containing 1,000 SNPs; thus retaining the realistic LD pattern between SNPs.

### Phenotype data

We assume each analysis unit contains 1, 2, 3 or 4 causal variants. The genomic position of causal variants are simulated to associate with genomic annotations. From previous enrichment analysis of 5 annotations we estimate enrichment of GWAS signals in these regions with odds ratios ranging from 3.70 to 6.02, with mean 4.74. The 5 annotations physically cover a total of 13.36% of the genome. For simplicity we create for each analysis unit 5 non-overlapping consective regions with total length constituting 13.36% of the unit of interest. Let $p_1$ and $p_0$ denote causal probability of SNPs inside and outside these regions respectively, 

\begin{align}
\gamma & = \frac{p_1/1-p_1}{p_0/1-p_0} \\
L & = [qp_1 + (1-q)p_0] \times N
\end{align}

where $\gamma$ is the mean odds ratio ($\gamma = 4.74$), $L$ is the number of causal variants in the region ($L = 1,2,3,4$), $N$ is total length of the region ($N=1000$), $q$ is proportion of annotation covered region ($q=0.1336$).

To simulate phenotypes we assume for a causal variant an effect size $\beta_j \sim N(0, \sigma^2)$. We generate phenotype from a linear model $y=X\beta + E, E \sim N(0, \sigma_a^2)$. Relative strength of $\sigma^2$ and $\sigma_a$ are determined by percentage of variance explained (PVE) by all genetic effects. We set PVE to 0.1.

### Caveats

Here we only have ~600 genotype samples to work with. Also we simulated quantitive phenotypes. The type of GWAS of interest here typically involves hundreds of thousands of samples with binary disease outcome. However we argue that in practice we work with summary statistics, which are typically z-scores whether they are from GWAS of disease phenotypes or quantitative traits. Although generated from ~600 sample, the observed effect sizes $\hat{\beta}$ should have similar "LD convoluted" structure as they are from larger samples. So our settings here should be good enough.

## Analysis

We use DAP-G to analyze the data. We run and compare two versions of DAP-G: one that uses the "oracle" prior from enrichment based simulation, one uses uniform priors.

## DSC benchmark

Simulate study is implemented in DSC framework. Input data are just matrices of genotypes.

### `zzz.dsc`

In [1]:
%save -f modules/zzz.dsc
%include modules/simulate_prior
%include modules/simulate_y
%include modules/fit
%include modules/evaluate

Cell content saved to modules/zzz.dsc

### `master.dsc`

In [2]:
%save -f master.dsc
#!/usr/bin/env dsc
%include modules/zzz

DSC:
    define:
        fit: dap, dapa
    run: simulate_prior * simulate_y * fit * evaluate
    exec_path: modules
    global:
        data_file: gtex-manifest.txt
        n_units: 10
    output: xh_grant

Cell content saved to master.dsc

### `simulate_prior.dsc`

In [3]:
%save -f modules/simulate_prior.dsc

simulate_prior: sim_prior.R + R(data = readRDS(dataset);
                           X = get_loci(data$X, N);
                           prior = get_prior(N, chunks, g, q))
  dataset: Shell{head -${n_units} ${data_file}}
  N: 1000
  chunks: 5
  g: 4.74
  q: 0.1336
  $X: X
  $prior: prior$prior
  $annotation: prior$annotation

Cell content saved to modules/simulate_prior.dsc

### `simulate_y.dsc`

In [4]:
%save -f modules/simulate_y.dsc
simulate_y: sim_y.py + Python(coef, y, residual_var, L = simulate(X, prior, n_signal, pve, amplitude))
  X: $X
  prior: $prior
  n_signal: 1, 2, 3, 4
  pve: 0.1
  amplitude: 0.6
  $y: y
  $coef: coef
  $L: L
  $residual_var: residual_var

Cell content saved to modules/simulate_y.dsc

### `fit.dsc`

In [5]:
%save -f modules/fit.dsc

dap: fit_dap.py + Python(posterior = dap_batch(X, y, cache, None, args))
  X: $X
  y: $y
  args: "-ld_control 0.20 --all"
  cache: file(DAP)
  $posterior: posterior

dapa(dap): fit_dap.py + Python(posterior = dap_batch(X, y, cache, prior, args))
  prior: $prior

Cell content saved to modules/fit.dsc

### `evaluate.dsc`

In [6]:
%save -f modules/evaluate.dsc

evaluate: evaluate_dap.py
  coef: $coef
  posterior: $posterior
  $is_recovered: list(is_recovered.items())
  $is_cs_true: list(is_cs_true.items())
  $is_top_true: list(is_top_true.items())
  $size: list(size.items())
  $purity: list(purity.items())

Cell content saved to modules/evaluate.dsc

## Simulation

### `sim_prior.R`

From equations above, assuming $L=1$ we derive:

\begin{align}
p_1 = \frac{\gamma p_0}{1 - p_0 + \gamma p_0} \\
N(q-1-\gamma q + \gamma)p_0^2 - (Nq - N - Nq\gamma + \gamma - 1)p_0 - 1= 0
\end{align}

We can solve this numerically in R, eg:

In [7]:
g = 4.74
N = 1000
q = 0.1336
foo = function(x) N * (q-1-g*q+g) * x^2 - (N*q-N-N*q*g+g-1) * x - 1
p0 = uniroot(foo, lower=0, upper=1, tol = .Machine$double.eps^0.8)$root
p1 = g * p0 / (1-p0+g*p0)
print(c(p0,p1))

[1] 0.000667518 0.003156156


In [8]:
# verify it:
print(p1/(1-p1) / (p0 / (1-p0)))
print(N * q * p1 + N * (1-q) * p0)

[1] 4.74
[1] 1


We should be good. I'll use this in my simulation code below.

In [9]:
%save -f modules/sim_prior.R
get_loci = function(X, N) {
    segs = floor(ncol(X) / N)
    lapply(1:segs, function(i) X[,i:(i+N-1)])
}
get_prior = function(N, chunks, g, q) {
    foo = function(x) N * (q-1-g*q+g) * x^2 - (N*q-N-N*q*g+g-1) * x - 1
    p0 = uniroot(foo, lower=0, upper=1, tol = .Machine$double.eps^0.8)$root
    p1 = g * p0 / (1-p0+g*p0)
    per_chunk_len = N * q / chunks
    n_bins = floor(N/chunks)
    annotated = unlist(lapply(1:chunks, function(i) ((i-1) * n_bins + 1):((i-1) * n_bins + per_chunk_len)))
    prior = rep(p0, N)
    prior[annotated] = p1                          
    list(prior=prior, annotation=annotated)                          
}

Cell content saved to modules/sim_prior.R

### `sim_y.py`

In [10]:
%save -f modules/sim_y.py
import numpy as np
import os
from pprint import pformat
from collections import OrderedDict
import time

class RegressionData:
    def __init__(self, X = None, Y = None, Z = None):
        # FIXME: check if inputs are indeed numpy arrays
        self.x_mean = self.y_mean = self.z_mean = None
        self.X = X
        self.Y = Y
        self.Z = Z
        if X is not None:
            np.random.seed(sum(X))

    def center_data(self):
        # for np.array: np.mean(Z, axis=0, keepdims=True)
        # for np.matrix, no keepdims argument
        if self.X is not None and self.x_mean is None:
            self.x_mean = np.mean(self.X, axis=0)
            self.X -= self.x_mean
        if self.Y is not None and self.y_mean is None:
            self.y_mean = np.mean(self.Y, axis=0)
            self.Y -= self.y_mean
        if self.Z is not None and self.z_mean is None:
            self.z_mean = np.mean(self.Z, axis=0)
            self.Z -= self.z_mean

    def __str__(self):
        return pformat(self.__dict__, indent = 4)

class UnivariateSimulator:
    def __init__(self, dim):
        self.size = dim
        self.pi0 = 0
        self.pis = []
        self.mus = []
        self.sigmas = []
        self.coef = []
        self.residual_variance = None
        
    def set_vanilla(self, amplitude):
        self.pis = [1]
        self.mus = [0]
        self.sigmas = [amplitude]
        
    def get_effects(self):
        '''
        beta ~ \pi_0\delta_0 + \sum \pi_i N(mu_i, sigma_i)
        '''
        sigmas = np.diag(self.sigmas)
        assert (len(self.pis), len(self.pis)) == sigmas.shape
        masks = np.random.multinomial(1, self.pis, size = self.size)
        mix = np.random.multivariate_normal(self.mus, sigmas, self.size)
        self.coef = np.sum(mix * masks, axis = 1) * np.random.binomial(1, 1 - self.pi0, self.size)

    def sparsify_effects(self, num_non_zero, prior):
        # FIXME: might overlap if some prior is very high; thus not gauranteed to have num_non_zero non-zeros
        ones = np.random.multinomial(num_non_zero, prior)
        self.coef = self.coef * ones
                
    def get_y(self, regression_data, pve = None, sigma = None):
        if sigma is None and pve is None:
            raise ValueError('Need one of sigma or pve.')
        if not (pve > 0 and pve < 1):
            raise ValueError(f'PVE has to be between 0 and 1, not {pve}.')
        if pve is not None:
            genetic_var = np.var(np.dot(regression_data.X, self.coef.T))
            pheno_var = genetic_var / pve
            self.residual_variance = pheno_var - genetic_var
        y = np.dot(regression_data.X, self.coef.T) + np.random.normal(0, np.sqrt(self.residual_variance), regression_data.X.shape[0])
        # y.reshape(len(y), 1)
        return y.T
    
def simple_lm(X, prior, c):
    reg = RegressionData(X)
    reg.center_data()
    eff = UnivariateSimulator(reg.X.shape[1])
    eff.set_vanilla(c['amplitude'])
    eff.get_effects()
    eff.sparsify_effects(c['n_signal'], prior)
    y = eff.get_y(reg, pve = c['pve'])
    return np.array(eff.coef).T, eff.residual_variance, y

def simulate(X, prior, n_signal, pve, amplitude):
    coef = {k:[] for k in X}
    residual_var = []
    y = {k:[] for k in X}
    c = { 'n_signal': n_signal, 'pve': pve, 'amplitude': amplitude}
    for k in X:
        tmp = simple_lm(X[k], prior, c)
        coef[k] = tmp[0]
        residual_var.append(tmp[1])
        y[k] = tmp[2]
    return coef, y, residual_var, {k: sum(coef[k]!=0) for k in coef}

Cell content saved to modules/sim_y.py

## Fine-mapping with DAP

### `fit_dap.py`

Below is example output for DAP-G:

```
Posterior expected model size: 0.500 (sd = 0.500)
LogNC = -0.30685 ( Log10NC = -0.133 )
Posterior inclusion probability

((1))              7492 6.68581e-05       0.000 1
((2))              7490 6.68581e-05       0.000 1
... 7 lines
((8))              7491 6.68046e-05       0.000 2
((9))              7483 6.68046e-05       0.000 2
((10))             7485 6.68046e-05       0.000 2
... 13 lines
((20))             7459 6.68046e-05       0.000 2
((21))             7482 6.67422e-05       0.000 -1
((22))             7489 6.67422e-05       0.000 -1
... other lines until below ...

Independent association signal clusters

     cluster         member_snp      cluster_pip      average_r2
       {1}              7            4.680e-04          0.951                 0.951   0.037
       {2}             13            8.685e-04          0.623                 0.037   0.623

```

In [11]:
%save -f modules/fit_dap.py
import subprocess
import pandas as pd
import numpy as np

def write_dap_full(x,y,prefix,r):
    names = np.array([('geno', i+1, f'group_{r}') for i in range(x.shape[1])])
    with open(f'{prefix}.data', 'w') as f:
        print(*(['pheno', 'pheno', f'group_{r}'] + list(np.array(y).ravel())), file=f)
        np.savetxt(f, np.hstack((names, x.T)), fmt = '%s', delimiter = ' ')
#     grid = '''         
#         0.0000  0.1000
#         0.0000  0.2000
#         0.0000  0.4000
#         0.0000  0.8000
#         0.0000  1.6000
#         '''
#     grid = '\n'.join([x.strip() for x in grid.strip().split('\n')])
#     with open(f'{prefix}.grid', 'w') as f:
#         print(grid, file=f)

def write_prior(prior, prefix):
    with open(f'{prefix}.prior', 'w') as f:
        f.write('\n'.join([f'{i+1}\t{p}' for i, p in enumerate(prior)]))

def run_dap_full(prior, prefix, args):
    cmd = ['dap-g', '-d', f'{prefix}.data', '-o', f'{prefix}.result', '--output_all'] + ' '.join(args).split()
    if prior is not None:
        cmd.extend(['-p', f'{prefix}.prior'])
    subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()    

def extract_dap_output(prefix):
    out = [x.strip().split() for x in open(f'{prefix}.result').readlines()]
    pips = []
    clusters = []
    still_pip = True
    for line in out:
        if len(line) == 0:
            continue
        if len(line) > 2 and line[2] == 'cluster_pip':
            still_pip = False
            continue
        if still_pip and (not line[0].startswith('((')):
            continue
        if still_pip:
            pips.append([line[1], float(line[2]), float(line[3]), int(line[4])])
        else:
            clusters.append([len(clusters) + 1, float(line[2]), float(line[3])])
    pips = pd.DataFrame(pips, columns = ['snp', 'snp_prob', 'snp_log10bf', 'cluster'])
    clusters = pd.DataFrame(clusters, columns = ['cluster', 'cluster_prob', 'cluster_avg_r2'])
    clusters = pd.merge(clusters, pips.groupby(['cluster'])['snp'].apply(','.join).reset_index(), on = 'cluster')
    return {'snp': pips, 'set': clusters}

def dap_single(x, y, prefix, r, prior, args):
    write_dap_full(x,y,prefix,r)
    if prior is not None:
        write_prior(prior, prefix)
    run_dap_full(prior, prefix, args)
    return extract_dap_output(prefix)


def dap_batch(X, Y, prefix, prior, *args):
    return dict([(k, dap_single(X[k], Y[k], f'{prefix}_condition_{k}', k, prior, args)) for k in X])

Cell content saved to modules/fit_dap.py

## Evaluate results
### `evaluate.py`

In [12]:
%save -f modules/evaluate_dap.py
def dap_summary(cluster, snp, coef):
    cluster = cluster.loc[cluster['cluster_prob'] > 0.95]
    # to return
    purity = cluster['cluster_prob'].tolist()
    cluster = [x.split(',') for x in cluster['snp']]
    signal_detected = sum(cluster, [])
    signal_expected = [f'{idx+1}' for idx, i in enumerate(coef) if i != 0]
    # to return
    size = [len(c) for c in cluster]
    snp = [snp.loc[snp['snp'].isin(c)] for c in cluster]
    top_snp = [s.loc[s['snp_prob'] == max(s['snp_prob'])]['snp'].tolist()[0] for s in snp]
    # to return
    is_top_true = [1 if x in signal_expected else 0 for x in top_snp]
    # to return
    is_recovered = [1 if x in signal_detected else 0 for x in signal_expected]
    # to return
    is_cs_true = [1 if len(set(x).intersection(signal_expected)) else 0 for x in signal_detected]
    return is_recovered, is_cs_true, is_top_true, size, purity

is_recovered = dict()
is_cs_true = dict()
is_top_true = dict()
size = dict()
purity = dict()
for k in posterior:
    is_recovered[k], is_cs_true[k], is_top_true[k], size[k], purity[k] = dap_summary(posterior[k]['set'], posterior[k]['snp'], coef[k])    

Cell content saved to modules/evaluate_dap.py